In [ ]:
from crawl import Crawler
from parse import find_all_safely
import pandas as pd
import numpy as np
import os
from setup_db import DB, setup_db, cursor_as_df, execute_to_df

In [ ]:
# db = setup_db()

In [ ]:
path = "test_db.db"
db = DB(path)
db.run_setup_scripts()
#db.insert_initial_data()


In [ ]:
# db.close()

In [ ]:
db.select_all("jobs")

In [ ]:
db.select_all("requests")

In [ ]:
db.show_tables()

In [ ]:
db.select_all("results")

SELECT 
request_id,
strftime('%d.%m.%Y',t2.time_created) as rideDate,
t2.start_city as departureCity,
t2.end_city as arrivalCity
FROM jobs as t1
left join requests t2 ON t1.job_id = t2.job_id
WHERE t1.active = 1


In [ ]:
params_from_db = db.c.execute('''SELECT 
t2.request_id,
strftime("%d.%m.%Y",t2.date) as rideDate,
t2.start_city as departureCity,
t2.end_city as arrivalCity
FROM jobs as t1
left join requests as t2 ON t1.job_id = t2.job_id
WHERE t1.active = 1''')
colnames = [i[0] for i in params_from_db.description]
values = params_from_db.fetchall()

In [ ]:
print(colnames)
print(values)

In [ ]:
constant_search_params = {
    "adult": "1",
    "_locale": "pl"
}


In [ ]:
values

In [ ]:
params_list = []
for single_params in values:
    params_list.append({"rideDate": single_params[1],
                       "departureCity": single_params[2],
                       "arrivalCity": single_params[3]})
    
params_list = [{**i, **constant_search_params} for i in params_list]
request_id_list = [i[0] for i in values]

In [ ]:
params_list

In [ ]:
current_params = params_list[1]
current_request_id = request_id_list[1]

In [ ]:
crawler = Crawler(current_params)
crawler.get_all()
results_pd_raw  = crawler.get_pandas_df()

In [ ]:
results_pd = results_pd_raw.copy()

In [ ]:
results_pd_raw

In [ ]:
class dfSanitizer:
    def __init__(self, df):
        self.df = df

#         self.sanitize_station_vec = np.vectorize(sanitize_station)
        self.sanitize_price_vec = np.vectorize(sanitize_price)
#         self.sanitize_time_vec = np.vectorize(sanitize_time)
    
    def sanitize_price(self, x):
        if x is None:
            return None

        try:
            stripped = re.sub("[^\d\,]", "", x) # remove all characters except numbers and commas

            if len(stripped)>10:
                return None

            stripped_comma = stripped.replace(",", ".")

            number = float(stripped_comma)

        except:
            return None

        return number
    

In [ ]:
a = dfSanitizer(1)
a

In [ ]:
a.sanitize_price_vec(10)

In [ ]:
def sanitize_price(x):
    if x is None:
        return None
    
    try:
        stripped = re.sub("[^\d\,]", "", x) # remove all characters except numbers and commas
        
        if len(stripped)>10:
            return None

        stripped_comma = stripped.replace(",", ".")
        
        number = float(stripped_comma)
    
    except:
        return None
    
    return number
    

In [ ]:
print(sanitize_price("65,94") == 65.94)
print(sanitize_price("65,a94") == 65.94)
print(sanitize_price("65,a94   ") == 65.94)
print(sanitize_price("65,56,4") is None)
print(sanitize_price(None) is None)
print(sanitize_price("54,") == 54.0)
print(sanitize_price(",54") == 0.54)
print(sanitize_price(43) is None)

In [ ]:
def sanitize_station(station):
    try:
        return station.lstrip().rstrip()
    except:
        return None

In [ ]:
print(sanitize_station("abc") == "abc")
print(sanitize_station("      abc") == "abc")
print(sanitize_station("abc   ") == "abc")
print(sanitize_station(1) is None)
print(sanitize_station(None) is None)

In [ ]:
def sanitize_time(x):
    if x is None:
        return None
    
    try:
        stripped = re.sub("[^\d\:]", "", x) # remove all characters except numbers and commas
        
        if len(stripped) != 5:
            return None

        if sum([i == ":" for i in stripped]) != 1:
            return None
        
    
    except:
        return None
    
    return stripped

In [ ]:
print(sanitize_time("09:50") == "09:50")
print(sanitize_time("09:50   ") == "09:50")
print(sanitize_time("09:50aa") == "09:50")
print(sanitize_time("9:50:") is None)
print(sanitize_time("950") is None)
print(sanitize_time(":") is None)
print(sanitize_time("9:50") is None)

In [ ]:

sanitize_station_vec = np.vectorize(sanitize_station)
sanitize_price_vec = np.vectorize(sanitize_price)
sanitize_time_vec = np.vectorize(sanitize_time)

In [ ]:
results_pd

In [ ]:
results_pd.price = sanitize_price_vec(results_pd.price)
results_pd.departure_station = sanitize_station_vec(results_pd.departure_station)
results_pd.arrival_station = sanitize_station_vec(results_pd.arrival_station)
results_pd.departure_time = sanitize_time_vec(results_pd.departure_time)
results_pd.arrival_time = sanitize_time_vec(results_pd.arrival_time)
results_pd["fully_booked"] = results_pd.price.isna()

In [ ]:
results_pd["rideDate"] = current_params["rideDate"]
results_pd["departure_datetime"] = results_pd.rideDate + " " + results_pd.departure_time
results_pd["departure_datetime"] = pd.to_datetime(results_pd["departure_datetime"])
results_pd["arrival_datetime"] = results_pd.rideDate + " " + results_pd.arrival_time
results_pd["arrival_datetime"] = pd.to_datetime(results_pd["arrival_datetime"])


In [ ]:
mask = results_pd["arrival_datetime"] < results_pd["departure_datetime"]
masked_df = results_pd[mask]

In [ ]:
results_pd.loc[mask, "arrival_datetime"] = masked_df.arrival_datetime + pd.DateOffset(days = 1)

In [ ]:
from datetime import datetime

In [ ]:
datetime.now()

In [ ]:
current_params

In [ ]:
results_pd["request_id"] = current_request_id
results_pd["time_created"] = datetime.now()
results_pd["departure_city"] = current_params["departureCity"]
results_pd["arrival_city"] = current_params["arrivalCity"]

In [ ]:
db.select_all("results")

In [ ]:

results_pd

In [ ]:
unique_stations = set(results_pd.departure_station.tolist() + results_pd.arrival_station.tolist())
unique_stations

In [ ]:
results = []
for search_params in params_list:
    crawler = Crawler(search_params)
    crawler.get_all()
    results.append(crawler.get_pandas_df())
    
results

In [ ]:
sanitizer.sanitized_df

In [ ]:
sanitizer